In [1]:
import numpy as np
import problema_espacio_estados as probee
import búsqueda_espacio_estados as búsqee
import copy
from tkinter import *
from tkinter import ttk
import random

In [2]:
def leerTxt(archivo):
    for line in archivo.readlines():
        if(line != "\n"):
            matriz =[[ ]]
            i = 0
            for k in range(len(line)):
                anterior=line[k-1]
                actual=line[k]
                if actual == "," and anterior == "]":
                    matriz.append([])
                    i = i+1

                if actual != "[" and actual != "]" and actual != "," and actual != "\n": 
                    matriz[i].append(int(actual))               
            array_1.append(matriz)
    archivo.close()
    
def solucion(matrizOr,solucion):
    for x in range(len(solucion)):
        i = int(solucion[x][34])
        j = int(solucion[x][46])
        matrizOr[i][j] = 0

In [3]:
class InterfazGrafica():
    def __init__(self,matriz,matrizSol):
        self.raiz = Tk()
        self.raiz.geometry('600x400')
        self.raiz.configure(bg = 'beige')
        self.raiz.resizable(width=False,height=False)
        self.raiz.title('Hitori')
        
        self.tinfo = Text(self.raiz, width=60, height=20)
        
        self.tinfo.pack(side=TOP)
        self.binfo = ttk.Button(self.raiz, text='Info', command=self.verinfo(matriz,matrizSol))
        
        self.binfo.pack(side=LEFT)
        self.bsalir = ttk.Button(self.raiz, text='Salir', command=self.raiz.destroy)
                                 
        self.bsalir.pack(side=RIGHT)
        self.raiz.mainloop()
    
    def verinfo(self,matriz,matrizSol):
       
        self.tinfo.delete("1.0", END)
        widt = len(matriz)
        height = len(matriz[0])
        info1 = matriz
        info2 = matrizSol
        # Construye una cadena de texto con toda la
        # información obtenida:
        
        texto_info = "Tamaño de la matriz: " + str(widt) + " x " + str(height) + "\n" 
        texto_info += "Matriz: \n" 
        for i in range(widt):
            for j in range (height):
                texto_info += " | "+str(int(info1[i][j]))
            texto_info += "\n"
        texto_info += "\nSolución: \n"
        for i in range(widt):
            for j in range (height):
                if(info2[i][j] != 0):
                    texto_info += " | "+str(int(info2[i][j]))
                else:
                    texto_info += " | N"
            texto_info += "\n"
        
        # Inserta la información en la caja de texto:
        
        self.tinfo.insert("1.0", texto_info)

In [4]:
def aplicabilidadVecinos(estado,i,j):
    widt = len(estado)
    height = len(estado[0])
    if(j-1 >= 0):
        if(estado[i][j - 1] == 0):
            return False
    if(j+1 <= height-1):
        if(estado[i][j + 1] == 0):
            return False
    if(i+1 <= widt-1):
        if(estado[i + 1][j] == 0):
            return False
    if(i-1 >= 0):
        if(estado[i - 1][j] == 0):
            return False
    return True
    
def repetido(estado,i,j):
    widt = len(estado)
    height = len(estado[0])
    for x in range(widt):
        if(estado[i][j] == estado[x][j] and estado[i][j] != 0 and i!=x):
            return True
    for y in range(height):
        if(estado[i][j] == estado[i][y] and estado[i][j] != 0 and j!=y):
            return True
    return False

def aplicable(estado):
    return aplicabilidadVecinos(estado,2,2) and repetido(estado,2,2)

def ponerNegro(estado,i,j):
    estado[i][j] = 0

def aplicar(estado):
    nuevo_estado = copy.deepcopy(estado)
    ponerNegro(nuevo_estado,2,2)
    return nuevo_estado

C22 = probee.Acción('Marcar el numero que esta en fila 2 y columna 2',aplicable,aplicar)

In [5]:
class cambiarCelda(probee.Acción):
    def __init__(self, i, j, height, widt):
        nombre = 'Marcar el numero que esta en fila {} y columna {}'.format(i,j)
        super().__init__(nombre)
        self.i = i
        self.j = j
        self.height = height
        self.widt = widt

    def aplicabilidadVecinos(self,estado,i,j,height,widt):
        if(j-1 >= 0):
            if(estado[i][j - 1] == 0):
                return False
        if(j+1 <= height-1):
            if(estado[i][j + 1] == 0):
                return False
        if(i+1 <= widt-1):
            if(estado[i + 1][j] == 0):
                return False
        if(i-1 >= 0):
            if(estado[i - 1][j] == 0):
                return False
        return True
    
    def repetido(self,estado,i,j,widt,height):
        for x in range(widt):
            if(estado[i][j] == estado[x][j] and estado[i][j] != 0 and i!=x):
                return True
        for y in range(height):
            if(estado[i][j] == estado[i][y] and estado[i][j] != 0 and j!=y):
                return True
        return False
    
    def es_aplicable(self,estado):
        return (self.aplicabilidadVecinos(estado,self.i,self.j,self.height,self.widt) and
                self.repetido(estado,self.i,self.j,self.widt,self.height))

    def ponerNegro(self,estado,i,j):
        estado[i][j] = 0

    def aplicar(self,estado):
        nuevo_estado = copy.deepcopy(estado)
        self.ponerNegro(nuevo_estado,self.i,self.j)
        return nuevo_estado

In [6]:
class Hitori(probee.ProblemaEspacioEstados):
    def __init__(self,matriz,widt,height):
        acciones = [cambiarCelda(i,j,height,widt) for i in range(widt) for j in range(height)]
        estado_inicial = matriz
        super().__init__(acciones,estado_inicial)
        self.matriz = matriz
        self.height = height
        self.widt = widt
    
    def celdaLibre(self,estado):
        pos = []
        for j in range(len(estado[0])):
            if estado[0][j] != 0:
                pos.append(0)
                pos.append(j)
                return pos
            
    def recorrer(self,estado):
        posIni = self.celdaLibre(estado)
        visitados = []
        self.recorrerConexos(posIni,visitados,estado,self.widt,self.height)
        return visitados
    
    def recorrerConexos(self,pos,visitados,estado,widt,height):
        i = pos[0]
        j = pos[1]
        visitados.append(pos)
        if(i+1 < widt):
            if(estado[i+1][j] != 0 and [i+1,j] not in visitados):
                pos = [i+1,j]
                self.recorrerConexos(pos,visitados,estado,widt,height)
        if(i-1 >= 0):
            if(estado[i-1][j] != 0 and [i-1,j] not in visitados):
                pos = [i-1,j]
                self.recorrerConexos(pos,visitados,estado,widt,height)
        if(j+1 < height):
            if(estado[i][j+1] != 0 and [i,j+1] not in visitados):
                pos = [i,j+1]
                self.recorrerConexos(pos,visitados,estado,widt,height)
        if(j-1 >= 0):
            if(estado[i][j-1] != 0 and [i,j-1] not in visitados):
                pos = [i,j-1]
                self.recorrerConexos(pos,visitados,estado,widt,height)
    
    def matrizConexa(self,estado,widt,height):
        visitados = self.recorrer(estado)
        celdasBlancas = len(visitados)
        celdasNegras = 0
        for i in range(widt):
            for j in range(height):
                if(estado[i][j] == 0):
                    celdasNegras = celdasNegras + 1
        if(celdasBlancas + celdasNegras == widt*height):
            return True
        return False
    
    def no_repetidoF(self,matriz):
        for i in range(self.widt):
            for j in range(self.height):                  
                for k in range(self.height):
                    if(matriz[i][j] == matriz[i][k] and j != k and matriz[i][j] != 0):
                        return False
        return True
    
    def no_repetidoC(self,matriz):
        for j in range(self.height):
            for i in range(self.widt):                  
                for k in range(self.widt):                
                    if(matriz[i][j] == matriz[k][j] and i != k and matriz[k][j] != 0):
                        return False
        return True
    
    def es_estado_final(self,estado):
        return self.matrizConexa(estado,self.widt,self.height) and self.no_repetidoF(estado) and self.no_repetidoC(estado)

In [7]:
array_1 = [ ]
archivo = open('ejemplos_prueba.txt','r')
leerTxt(archivo)

In [9]:
matriz=array_1[8]
estadoIni = copy.deepcopy(matriz)
estadoFinal=copy.deepcopy(matriz)

widt = len(estadoIni)
height = len(estadoIni[0])

In [10]:
Hitori_marcar_celdas = Hitori(estadoIni,widt,height)

In [11]:
C22.es_aplicable(estadoIni)

True

In [12]:
Hitori_marcar_celdas.es_estado_final(C22.aplicar(estadoIni))

False

In [13]:
Hitori_marcar_celdas.es_estado_final(estadoIni)

False

In [14]:
b_profundidad = búsqee.BúsquedaEnProfundidad(detallado=False)

In [15]:
b_anchura = búsqee.BúsquedaEnAnchura(detallado=False)

In [16]:
b_óptima = búsqee.BúsquedaÓptima()

In [17]:
%%timeit -n1 -r1
b_profundidad.buscar(Hitori_marcar_celdas)

1.46 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [18]:
%%timeit -n1 -r1
b_anchura.buscar(Hitori_marcar_celdas)

10.3 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [19]:
cambio = b_profundidad.buscar(Hitori_marcar_celdas)

In [20]:
solucion(estadoFinal,cambio)

In [21]:
interfaz = InterfazGrafica(estadoIni,estadoFinal)

In [22]:
%%timeit -n1 -r1
b_óptima.buscar(Hitori_marcar_celdas)

10.8 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
